In [ ]:
!nvidia-smi

Thu Apr 22 00:18:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook, notebook

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
device = torch.device("cuda:0")

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
torch.cuda.is_available()

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from gluonnlp.data import SentencepieceTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
BASE_PATH = '/gdrive/MyDrive/etc/will'
# data_2019 = glob.glob(BASE_PATH + '/*') # + '*'

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
df = pd.read_csv(BASE_PATH+'/210422_second.csv', index_col=[0], encoding="utf-8", dtype=np.object) #dtype=np.object)
df.head(3)

In [ ]:
df['n100'].value_counts().plot(kind = 'bar')
print(df.groupby('n100').size().reset_index(name = 'count'))

In [ ]:
g2 = df.groupby('n10')
g2.count()

In [ ]:
tdf = df.copy()

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(tdf[['input']], tdf['n10'],
                                          test_size = 0.2,
                                          shuffle=True,
                                          stratify=tdf['n10'],
                                          random_state=2013)
print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))

In [ ]:
trainY.value_counts().plot(kind = 'bar')
print(trainY.size)

In [ ]:
# 문장 추출
sentences = trainX['input'].values
# sentences[:10]
train_dataset = pd.DataFrame({'sentence': sentences, 'label': trainY}, columns=['sentence', 'label'])

# 문장 추출
sentences = testX['input'].values
test_dataset = pd.DataFrame({'sentence': sentences, 'label': testY}, columns=['sentence', 'label'])
sentences[:10]

In [ ]:
dataset_train = train_dataset.values
dataset_test = test_dataset.values

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
# tokenizer = get_tokenizer()
# bertmodel, vocab = get_pytorch_kobert_model()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = []
        self.labels = []
        
        for i in dataset:
            try:
                self.sentences.append(transform([i[sent_idx]]))
                self.labels.append(np.int32(i[label_idx]))
            except Exception as e:
                print(e)
                print(len(self.sentences), len(self.labels))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 128 #128
batch_size = 2 #cuda out of memory ...
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
NUM_LABELS = 10

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, #768 #뭐여 이게.....뭔 조화여 dim이 달라지구로
                 #https://www.google.com/search?sxsrf=ALeKk01zaV-xIa8Z0CUMGLodIa4vl6Z9fg%3A1616147579994&lei=e3RUYIKtPMizmAXPr7D4BA&q=Mat1%20dim%201%20must%20match%20mat2%20dim%200&ved=2ahUKEwjC1qPzirzvAhXIGaYKHc8XDE8QsKwBKAB6BAgUEAE&biw=832&bih=720
                 num_classes=NUM_LABELS,     #9,#num_labels
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.25).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
from time import gmtime, strftime
START_TIME = strftime("%Y_%m_%d_%H_%M_%S", gmtime())

SAVE_PATH = BASE_PATH+"/bert_model_{}.pth".format(START_TIME)

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    torch.save(model.state_dict(), SAVE_PATH)
    print("Model saved at {}".format(SAVE_PATH))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))